Run, develop and demonstrate a pipeline for searching for potential lensed supernovae.

Steps:
* Download public ZTF alerts distributed any particular day.
* Filter these based on the minimal criteria of potential detections.
* Obtain redshift information from number of sources: masterlens, NED, SDSS photo-z, LS photo-z
* Fit SALT2 model on best redshift estimate, return estimated absolute peak magnitude
* Plot lightcurve fit + links to slack

Note that a full alert archive, with a size of up to 20 GB will be downloaded. Files are retrieved from:
https://ztf.uw.edu/alerts/public/
and will be put in the notebook directory. For trial runs, choose a day with less alerts (e.g. 20210516)

The code assumes that the Ampel libraries are installed. Three steps are needed for this:
* Start a local MongoDB instance (with default connections).
* Download AMPEL v 0.8 core, ZTF and ampel-hu-astro libraries.
* Create a default ampel configuration file through running 'ampel-config build > ampel_conf.yaml'.

In [ ]:
import sys, os
# As a convenience we use utils from ampel-notebooks to retrieve data
# Relative path assumes ampel-notebooks and ampel-hu-astro installed in same dir
sys.path.append(os.getcwd() + "/../../ampel-notebooks/")
from utils import get_ampel_config, get_ipac_tarball

In [ ]:
%load_ext ampel_quick_import
%qi DecentFilter DevAmpelContext

In [ ]:
# Select date of query with format e.g. '20210610'
channel = 'GeneralLens'
ampel_conf_path = '/home/jnordin/github/ampelpip8/ampel_conf_3bd852.yaml'
date = '20211001'
ITER_MAX = 5000
# Valid for 5000 alerts from 2021-10-01
tar_checksum = "ad6cb6e9cb0bf9ab0311dda03ac7aec0ac2ee12358f20754810069c3a29aa616"

In [ ]:
AMPEL_CONF = get_ampel_config(fallback=ampel_conf_path)
# Should download into other directory. 
ALERT_ARCHIVE = get_ipac_tarball(
    'https://ztf.uw.edu/alerts/public/ztf_public_{}.tar.gz'.format(date),
    alerts=ITER_MAX,
    sha256= tar_checksum,
)
AMPEL_CONF, ALERT_ARCHIVE

In [ ]:
ctx = DevAmpelContext.load(
    config = AMPEL_CONF,
    db_prefix='AmpelTest',
    purge_db=True
)

In [ ]:
ctx.add_channel(
    name=channel,
    access=['ZTF', 'ZTF_PUB', 'ZTF_PRIV']
)

In [ ]:
# These are configuration parameters for the initial alert filter step. Changing these will accept more/less
# alerts.
filter_config = {
    'min_ndet': 6,
    'min_tspan': 5,
    'max_tspan': 50,
    'min_archive_tspan': 5,
    'max_archive_tspan': 50,    
    'min_rb': 0.3,
    'min_drb': 0.995,
    'max_fwhm': 5.5,
    'min_gal_lat': 14,
    'ps1_sgveto_rad': 1,
    'ps1_sgveto_th': 0.8,
    'ps1_confusion_rad': 3,
    'ps1_confusion_sg_tol': 0.1
}

In [ ]:
# Will use NED for spectroscopic redshifts.
cat_conf = {
    'catalogs' : {
        'SDSS_spec' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'catq_kwargs' : {
                'ra_key' : 'ra',
                'dec_key' : 'dec'
                },
            'keys_to_append' : ['z', 'bptclass', 'subclass'],
            'pre_filter' : None,
            'post_filter' : None,
        },
        'NEDz' : {
            'use' : 'catsHTM',
            'rs_arcsec' : 10.0,
            'catq_kwargs' : {
                'ra_key' : 'RA',
                'dec_key' : 'Dec'
                },
            'keys_to_append' : ['ObjType', 'Velocity', 'z'],
            'pre_filter' : None,
            'post_filter' : None,
        },
        'NEDz_extcats' : {
            'use' : 'extcats',
            'rs_arcsec' : 60.0,
            'catq_kwargs' : {
                'ra_key' : 'RA',
                'dec_key' : 'Dec'
                },
            'keys_to_append' : ['ObjType', 'Velocity', 'z'],
            'pre_filter' : None,
            'post_filter' : {'z': {'$lte': 0.03, '$gte' : 0.002}}
        },        
        'GLADEv23' : {
            'use' : 'extcats',
            'catq_kwargs' : {
                'ra_key' : 'RA',
                'dec_key' : 'dec' },
            'rs_arcsec' : 10,
            'keys_to_append' : ['z', 'dist', 'dist_err', 'flag1', 'flag2', 'flag3'],
        },
    }
}

In [ ]:
# We with to digest available redshift to one used for lightcurve template
# the "Ampel" z weighs available matches and returns the mean of the most accurate ones (if available)
ampelz_conf = {
    "max_redshift_category" : 6,
    "t2_dependency": [
        {
            "unit": "T2CatalogMatch",
            "config": cat_conf,
            "link_override": {
                'filter': 'PPSFilter', 'sort': 'jd', "select": "first"
                }
        },
    ]
}

In [ ]:
# Finally, we perform a salt-fit using the aggregated data and the redshift from above
sncosmo_conf = {
    "sncosmo_model_name" : "salt2",
    "redshift_kind" : 'T2DigestRedshifts',
    "max_ampelz_group" : 7,      # For this purpose we use any available redshift
    "plot_dir": "/home/jnordin/tmp/ztfsalt",
    "t2_dependency": [
        {
            "unit": "T2DigestRedshifts",
            "config": ampelz_conf,
        },
    ]
   
}

In [ ]:
ac = ctx.new_context_unit(
    unit = "AlertConsumer",
    process_name = "AP_test",
    iter_max = ITER_MAX,
    log_profile = os.environ.get('log_profile', 'debug'),
    shaper = "ZiDataPointShaper",
    compiler_opts = "ZiCompilerOptions",
    supplier = {
        "unit": "ZiAlertSupplier",
        'config': {
            'deserialize': "avro",
            'loader': {
                'unit': 'TarAlertLoader',
                'config': {'file_path': ALERT_ARCHIVE}
            }
        }
    },
    directives = [
        {
            "channel": channel, 
            "filter": {
                "unit": "SimpleDecentFilter", 
                "config": filter_config,
                "on_stock_match": 'bypass',
            },
            "ingest": {
                "mux": {
                    "unit": "ZiMongoMuxer",
                    "combine": [
                        {
                            "unit": "ZiT1Combiner",
                            "state_t2": [
                                {'unit': 'T2LightCurveSummary'},
                                {
                                    "unit": "T2DigestRedshifts",
                                    "config": ampelz_conf,
                                },
                                {
                                    "unit": "T2RunSncosmo",
                                    "config": sncosmo_conf,
                                },
                            ]
                        }
                    ],
                    "insert": {
                        "point_t2": [
                            {
                                'unit': 'T2CatalogMatch',
                                'config': cat_conf,
                                "ingest": {
                                    'filter': 'PPSFilter', 'sort': 'jd', 'select': 'first'
                                }
                            },
                        ]
                    }
                }
            }
        },
    ]
)

In [ ]:
n = ac.run()

In [ ]:
print(f"{n} alerts processed for channel {channel}")

In [ ]:
t2w = ctx.new_context_unit(
    unit = 'T2Worker',
    process_name = 'T2Processor_test',
    log_profile = os.environ.get('log_profile', 'default')
)

In [ ]:
t2w.run()

In [ ]:
table_config =                         {
                                        'table_schema' : {
                                            'T2DigestRedshifts': {
                                                'Ampel z' : ['ampel_z'],
                                                'Ampel z group' : ['group_z_nbr'],
                                            },
                                            'T2RunSncosmo' : {
                                                'Peak brightness (abs mag)' : ['fit_metrics','restpeak_model_absmag_B'],
                                                'SALT2 X1' : ['sncosmo_result','paramdict','x1'],
                                                'SALT2 X1 (err)' : ['sncosmo_result','errors','x1'],
                                                'SALT2 Color' : ['sncosmo_result','paramdict','c'],
                                                'SALT2 Color (err)' : ['sncosmo_result','errors','c'],
                                                'Pulls around peak' : ['fit_metrics','absmean_peak_pull'],
                                                'Det. around peak' : ['fit_metrics','nbr_peak_pulls'],
                                            },
                                        },
                                        'transient_table_schema' : {
                                            'T2CatalogMatch': {
                                                'SDSS spec z' : ['SDSS_spec','z'],
                                                "NED z" : ['NEDz','z'],
                                                "Glade z" : ['GLADEv23','z'],
                                                "Void radius (Mpc)" : ['voidAllPan','radius'],
                                            },
                                        },
                                        'include_stock' : True,
                                        'include_channels' : True,
                                        'fmt' : 'csv',
                                        'local_path' : '/home/jnordin/tmp'
                                    }

In [ ]:
t2_filter =  {'all_of':
              [
                {
                    'unit': 'T2RunSncosmo',
                    'match': {
                        'fit_metrics.restpeak_model_absmag_B': {"$lt":-19.0},
                        'sncosmo_result.paramdict.x1': {"$lt":4.5},
                        'sncosmo_result.paramdict.x1': {"$gt":-4.5},
                        'fit_metrics.absmean_peak_pull': {"$lt":10},
                        'fit_metrics.nbr_peak_pulls': {"$gt":3},
                    }
                },
                {
                    'unit': 'T2DigestRedshifts',
                    'match': {
                        'ampel_z': {"$lt":0.6},
                        'group_z_nbr': {"$lt":4},
                    }
                },
             ]
    }

In [ ]:
t3p = ctx.new_context_unit(
    unit = "T3Processor",
    process_name = "T3Processor_test",
    log_profile = "default",
    session = [
        {"unit": "T3SessionLastRunTime", "config": {"fallback": {"days": -90}}},
        {"unit": "T3SessionAlertsNumber"}
    ],
    supply = {
        "unit": "T3DefaultSupplier",
        "config": {
            "select": {
                "unit": "T3FilteringStockSelector",
                "config": {
                    "created": {"after": {"match_type": "time_delta", "days": -40}},
                    "updated": {"after": {"match_type": "time_delta", "days": -40}},
                    "channel": channel,
                    'tag': {'with': "ZTF", 'without': 'HAS_ERROR'},
                    't2_filter': t2_filter,
                }
            },
            "load": {
                "unit": "T3SimpleDataLoader",
                "config": {
                    "directives": ["STOCK", "DATAPOINT", "COMPOUND", "T2DOC"],
                }
            },
            "complement": [
                {"unit": "T3LogsAppender"}
            ]
        }
    },
    stage = {
        "unit": "T3SimpleStager",
        "config": {
            "execute": [
                {"unit": "TransientInfoPrinter"},
                {
                    "unit": "TransientTablePublisher",
                    "config": table_config
                }
            ]
        }
    }
)

In [ ]:
t3p.run()